This notebook is the first of a series of various data analyses in which I try to better understand the United States' opioid epidemic. For background on this public health crisis: https://www.hhs.gov/opioids/about-the-epidemic/

The first dataset I will examine is the State of Connectcut's "Accidental Drug Related Deaths 2012-2016" https://catalog.data.gov/dataset/accidental-drug-related-deaths-january-2012-sept-2015 

The corresponding csv file can be found in the directory.

In [53]:
#first, let's load the data and see its structure
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import *
import shapefile
dataframe = pd.read_csv('Accidental_Drug_Related_Deaths__2012-2016.csv')

In [2]:
print(dataframe.shape)

(3045, 32)


In [3]:
#let's see what the headers look like 
print(dataframe.columns)

Index(['CaseNumber', 'Date', 'Sex', 'Race', 'Age', 'Residence City',
       'Residence State', 'Residence County', 'Death City', 'Death State',
       'Death County', 'Location', 'DescriptionofInjury', 'InjuryPlace',
       'ImmediateCauseA', 'Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone',
       'Oxymorphone', 'EtOH', 'Hydro-codeine', 'Benzodiazepine', 'Methadone',
       'Amphet', 'Tramad', 'Morphine (not heroin)', 'Other', 'Any Opioid',
       'MannerofDeath', 'AmendedMannerofDeath', 'DeathLoc'],
      dtype='object')


As we see, there are a lot of different angles from which we can appraoch this data. Both the spatial and temporal elements can help provide more insight into how the epidemic is affecting the state of Connecticut. 

In [4]:
#from a high level, let's first see how the number of heroin-related deaths has changed over the course of the 4 years. 
#we can construct dataframes for each year and save them for later use

#converting the date column into a more readable format 
#for date in dataframe['Date']: 
 #   print(date)

clean_dates = []

df_2012 = dataframe[dataframe['Date'].str.contains('2012')==True]
df_2013 = dataframe[dataframe['Date'].str.contains('2013')==True]
df_2014 = dataframe[dataframe['Date'].str.contains('2014')==True]
df_2015 = dataframe[dataframe['Date'].str.contains('2015')==True]
df_2016 = dataframe[dataframe['Date'].str.contains('2016')==True]



In [5]:
#We see 1652 heroin-related deaths in the entire dataset
print(dataframe['Heroin'].value_counts()['Y'])


1652


In [6]:
#Let's dig further into how many heroin-related deaths have occured on a year-by-year basis. 


print("Year\tHeroin-Related Deaths")
print("2012\t"+str(df_2012['Heroin'].value_counts()['Y']))
print("2013\t"+str(df_2013['Heroin'].value_counts()['Y']))
print("2014\t"+str(df_2014['Heroin'].value_counts()['Y']))
print("2015\t"+str(df_2015['Heroin'].value_counts()['Y']))
print("2016\t"+str(df_2016['Heroin'].value_counts()['Y']))


Year	Heroin-Related Deaths
2012	174
2013	256
2014	325
2015	415
2016	482


As we can see, the number of heroin-related deaths in the state of Connecticut has increased steadily in the past four years. We can use this as a baseline to dig deeper. 

Let's now try to visualize the given data geospatially. We can use external Census data to construct a heatmap of the state of Connecticut colored by the number of heroin-related deaths at the zip-code level. We can then see how this evolves year-to-year. 

For this, we can use a zip-code database: http://federalgovernmentzipcodes.us/

In [7]:
#Load the data 
zip_codes = pd.read_csv('free-zipcode-database.csv', dtype = {"RecordNumber": int, "Zipcode": str}, low_memory=False)

In [54]:
#pull the zip codes for CT only 
zip_codes_ct = zip_codes[zip_codes['State'].str.contains('CT') == True]
#throw out the rest of the data
zip_codes_ct = zip_codes_ct[['Zipcode', 'City']]

#joining the data on the city name to obtain the zip code for each city
#note: this isn't wholly accurate since one city may have multiple zip codes, but we will ignore this for simplicity
dataframe_zip = pd.merge(dataframe, 
                zip_codes_ct.drop_duplicates('City'), left_on='Death City', right_on='City', 
                how='left')

In [93]:
#as it turns out, GIS is more involved than I initially thought. I will come back to creating the map
#We can at least see which cities have had the greatest number of heroin-related deaths

df = dataframe_zip.groupby('Zipcode').count()
print(df[['Heroin']].sort_values('Heroin', ascending=False)[0:5])



         Heroin
Zipcode        
06101       199
06501       125
06701       121
06601       110
06050        79
